# Federated 2d image classification with MONAI

## Introduction

This tutorial shows how to deploy in Fed-BioMed the 2d image classification example provided in the project MONAI (https://monai.io/):

https://github.com/Project-MONAI/tutorials/blob/master/2d_classification/mednist_tutorial.ipynb

Being MONAI based on PyTorch, the deployment within Fed-BioMed follows seamlessy the same general structure of general PyTorch models. 

Following the MONAI example, this tutorial is based on the MedNIST dataset:

https://github.com/Project-MONAI/MONAI/blob/master/examples/notebooks/mednist_tutorial.ipynb.

## Creating MedNIST nodes

MedNIST provides an artificial 2d classification dataset created by gathering different medical imaging datasets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license.

To proceed with the tutorial, we created an iid partitioning of the MedNIST dataset between 3 clients. Each client has 3000 image samples for each class. The training partitions are availables at the following link:

https://drive.google.com/file/d/1vLIcBdtdAhh6K-vrgCFy_0Y55dxOWZwf/view

The dataset owned by each client has structure:


└── client_*/

    ├── AbdomenCT/
    
    └── BreastMRI/
    
    └── CXR/
    
    └── ChestCT/
    
    └── Hand/
    
    └── HeadCT/   

To create the federated dataset, we follow the standard procedure for node creation/population of Fed-BioMed. 
After activating the fedbiomed network with the commands

`source ./scripts/fedbiomed_environment network`

and 

`./scripts/fedbiomed_run network`

we create a first node by using the commands

`source ./scripts/fedbiomed_environment node`

`./scripts/fedbiomed_run node start`

We then poulate the node with the data of first client:

`./scripts/fedbiomed_run node add`

We select option 3 (images) to add MedNIST partition of client 1, by just picking the folder of client 1. 
We can further check that the data has been added by executing `./scripts/fedbiomed_run node list`

Following the same procedure, we create the other two nodes with the datasets of client 2 and client 3 respectively.


## Running Fed-BioMed Researcher

We are now ready to start the reseracher enviroment with the command `source ./scripts/fedbiomed_environment researcher`, and open the Jupyter notebook. 

In [1]:
%load_ext autoreload
%autoreload 2

We can first quesry the network for the mednist dataset. In this case, the nodes are sharing the respective partitions unsing the same tag `mednist`:

In [2]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

2022-01-05 09:31:00,876 fedbiomed INFO - Component environment:
2022-01-05 09:31:00,877 fedbiomed INFO - - type = ComponentType.RESEARCHER
2022-01-05 09:31:01,439 fedbiomed INFO - Messaging researcher_6614eef6-5331-401a-b4de-4e2a69e51c81 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x103f98760>
2022-01-05 09:31:01,506 fedbiomed INFO - Listing available datasets in all nodes... 
2022-01-05 09:31:01,514 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d / DEBUG - Message received: {'researcher_id': 'researcher_6614eef6-5331-401a-b4de-4e2a69e51c81', 'command': 'list'}
2022-01-05 09:31:01,517 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / DEBUG - Message received: {'researcher_id': 'researcher_6614eef6-5331-401a-b4de-4e2a69e51c81', 'command': 'list'}
2022-01-05 09:31:01,520 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / DEBUG - Message received: {'researcher_id': 'res

{'node_d562b808-8ed3-401b-8222-494283894e5d': [{'name': 'mednist',
   'data_type': 'images',
   'tags': ['mednist'],
   'description': 'bla',
   'shape': [16954, 3, 64, 64]}],
 'node_b4095fb0-5dac-49f5-9af7-a478f8942a4a': [{'name': 'mednist',
   'data_type': 'images',
   'tags': ['mednist'],
   'description': 'bla',
   'shape': [18000, 3, 64, 64]}],
 'node_de591a20-68ff-4697-8abb-9daccd955df2': [{'name': 'mednist',
   'data_type': 'images',
   'tags': ['mednist'],
   'description': 'bla',
   'shape': [18000, 3, 64, 64]}]}

## Create an experiment to train a model on the data found

The code for network and data loader of the MONAI tutorial can now be deployed in Fed-BioMed.
We first import the necessary modules from `fedbiomed` and `monai` libraries:

In [3]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/class_export_mnist.py'

In [4]:
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import decollate_batch
from monai.metrics import ROCAUCMetric
from monai.networks.nets import DenseNet121
from monai.transforms import (
    Activations,
    AddChannel,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    EnsureType,
)
from monai.utils import set_determinism

We can now define the training plan. Note that we can simply use the standard `TorchTrainingPlan` natively provided in Fed-BioMed. We reuse the `MedNISTDataset` data loader defined in the original MONAI tutorial, which is returned by the method `training_data`, which also implements the data parsing from the nodes `dataset_path`. Following the MONAI tutorial, the model is the `DenseNet121`.

In [5]:
%%writefile "$model_file"

import os
import numpy as np
import torch
import torch.nn as nn
from fedbiomed.common.torchnn import TorchTrainingPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import decollate_batch
from monai.metrics import ROCAUCMetric
from monai.networks.nets import DenseNet121
from monai.transforms import (
    Activations,
    AddChannel,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    EnsureType,
)
from monai.utils import set_determinism


# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, kwargs):
        super(MyTrainingPlan, self).__init__()
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["import numpy as np",
                "import os",
                "from torch.utils.data import DataLoader",
                "from monai.apps import download_and_extract",
                "from monai.config import print_config",
                "from monai.data import decollate_batch",
                "from monai.metrics import ROCAUCMetric",
                "from monai.networks.nets import DenseNet121",
                "from monai.transforms import ( Activations, AddChannel, AsDiscrete, Compose, LoadImage, RandFlip, RandRotate, RandZoom, ScaleIntensity, EnsureType, )",
                "from monai.utils import set_determinism",]
        self.add_dependency(deps)
         
        self.num_class =  kwargs['num_class']  
        self.model = DenseNet121(spatial_dims=2, in_channels=1,
                    out_channels = self.num_class)
        
        self.loss_function = torch.nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    class MedNISTDataset(torch.utils.data.Dataset):
            def __init__(self, image_files, labels, transforms):
                self.image_files = image_files
                self.labels = labels
                self.transforms = transforms

            def __len__(self):
                return len(self.image_files)

            def __getitem__(self, index):
                return self.transforms(self.image_files[index]), self.labels[index]
    
    def parse_data(self, path):
        print(self.dataset_path)
        class_names = sorted(x for x in os.listdir(path)
                     if os.path.isdir(os.path.join(path, x)))
        num_class = len(class_names)
        image_files = [
                        [
                            os.path.join(path, class_names[i], x)
                            for x in os.listdir(os.path.join(path, class_names[i]))
                        ]
                        for i in range(num_class)
                      ]
        
        return image_files, num_class
    
    def training_data(self, batch_size = 48):
        self.image_files, num_class = self.parse_data(self.dataset_path)
        
        if self.num_class!=num_class:
                raise Exception('number of available classes does not match declared classes')
        
        num_each = [len(self.image_files[i]) for i in range(self.num_class)]
        image_files_list = []
        image_class = []
        
        for i in range(self.num_class):
            image_files_list.extend(self.image_files[i])
            image_class.extend([i] * num_each[i])
        num_total = len(image_class)
        
        
        length = len(image_files_list)
        indices = np.arange(length)
        np.random.shuffle(indices)

        val_split = int(1. * length) 
        train_indices = indices[:val_split]

        train_x = [image_files_list[i] for i in train_indices]
        train_y = [image_class[i] for i in train_indices]


        train_transforms = Compose(
            [
                LoadImage(image_only=True),
                AddChannel(),
                ScaleIntensity(),
                RandRotate(range_x=np.pi / 12, prob=0.5, keep_size=True),
                RandFlip(spatial_axis=0, prob=0.5),
                RandZoom(min_zoom=0.9, max_zoom=1.1, prob=0.5),
                EnsureType(),
            ]
        )

        val_transforms = Compose(
            [LoadImage(image_only=True), AddChannel(), ScaleIntensity(), EnsureType()])

        y_pred_trans = Compose([EnsureType(), Activations(softmax=True)])
        y_trans = Compose([EnsureType(), AsDiscrete(to_onehot=num_class)])

        print(
            f"Training count: {len(train_x)}")
        
        
        train_ds = self.MedNISTDataset(train_x, train_y, train_transforms)
        train_loader = torch.utils.data.DataLoader(
            train_ds, batch_size, shuffle=True)
        
        return train_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = self.loss_function(output, target)
        return loss


Writing /Users/mlorenzi/works/temp/fedbiomed/var/tmp/tmp8_zt0d5r/class_export_mnist.py


We now set the model and training parameters. Note that we use only 1 epoch for this experiment, and perform the training on ~26% of the locally available training data.

In [6]:
model_args = {'num_class':6,}

training_args = {
    'batch_size': 20, 
    'lr': 1e-5, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum':250 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

The experiment can be now defined, by providing the `mednist` tag, and running the local training on nodes with model defined in `model_path`, standard `aggregator` (FedAvg) and `client_selection_strategy` (all nodes used). Federated learning is going to be perfomed through 3 optimization rounds.

In [7]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['mednist']
rounds = 3

exp = Experiment(tags=tags,
                 #clients=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None
                )

2022-01-05 09:31:14,352 fedbiomed INFO - Searching dataset with data tags: ['mednist'] for all nodes
2022-01-05 09:31:14,359 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / DEBUG - Message received: {'researcher_id': 'researcher_6614eef6-5331-401a-b4de-4e2a69e51c81', 'tags': ['mednist'], 'command': 'search'}
2022-01-05 09:31:14,360 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / DEBUG - Message received: {'researcher_id': 'researcher_6614eef6-5331-401a-b4de-4e2a69e51c81', 'tags': ['mednist'], 'command': 'search'}
2022-01-05 09:31:14,363 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d / DEBUG - Message received: {'researcher_id': 'researcher_6614eef6-5331-401a-b4de-4e2a69e51c81', 'tags': ['mednist'], 'command': 'search'}
2022-01-05 09:31:24,365 fedbiomed INFO - Node selected for training -> node_b4095fb0-5dac-49f5-9af7-a478f8942a4a
2022-01-05 09:31:24,366 fedbiomed INFO - Node selected for training -> node_de591a20-68ff-

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the clients

In [8]:
exp.run()

2022-01-05 09:31:26,660 fedbiomed INFO - Sampled nodes in round 0 ['node_b4095fb0-5dac-49f5-9af7-a478f8942a4a', 'node_de591a20-68ff-4697-8abb-9daccd955df2', 'node_d562b808-8ed3-401b-8222-494283894e5d']
2022-01-05 09:31:26,661 fedbiomed INFO - Send message to node node_b4095fb0-5dac-49f5-9af7-a478f8942a4a - {'researcher_id': 'researcher_6614eef6-5331-401a-b4de-4e2a69e51c81', 'job_id': 'a538d4c6-01bc-48b5-beee-5b4db6b8e01f', 'training_args': {'batch_size': 20, 'lr': 1e-05, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'num_class': 6}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/05/my_model_43380103-454c-42a3-bc34-13927bc21e33.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/05/my_model_c4264cc8-3562-4a64-bbf6-ed9ba20b5b42.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_b4095fb0-5dac-49f5-9af7-a478f8942a4a': ['dataset_acb23d50-9435-4549-9c84-457ed3ed462c']}}
2022-01-05 09:31:26,661 fedbiomed DEBUG - resea

2022-01-05 09:31:29,230 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / DEBUG - Dataset_path/Users/mlorenzi/works/temp/MedNIST/client_2
2022-01-05 09:31:29,231 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / DEBUG - Dataset_path/Users/mlorenzi/works/temp/MedNIST/client_1
2022-01-05 09:31:30,470 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d / INFO - Train Epoch: 1 [0/16954 (0%)]	Loss: 1.851112
2022-01-05 09:31:30,498 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / INFO - Train Epoch: 1 [0/18000 (0%)]	Loss: 1.790576
2022-01-05 09:31:30,521 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / INFO - Train Epoch: 1 [0/18000 (0%)]	Loss: 1.894149
2022-01-05 09:31:41,191 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / INFO - Train Epoch: 1 [200/18000 (1%)]	Loss: 1.672583
2022-01-05 09:31:41,227 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a

2022-01-05 09:34:59,479 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d / INFO - Train Epoch: 1 [3400/16954 (20%)]	Loss: 0.988173
2022-01-05 09:35:11,444 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / INFO - Train Epoch: 1 [3600/18000 (20%)]	Loss: 0.461884
2022-01-05 09:35:12,286 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / INFO - Train Epoch: 1 [3600/18000 (20%)]	Loss: 0.744591
2022-01-05 09:35:12,877 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d / INFO - Train Epoch: 1 [3600/16954 (21%)]	Loss: 0.784376
2022-01-05 09:35:23,316 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / INFO - Train Epoch: 1 [3800/18000 (21%)]	Loss: 0.626961
2022-01-05 09:35:24,145 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / INFO - Train Epoch: 1 [3800/18000 (21%)]	Loss: 0.577616
2022-01-05 09:35:24,780 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d /

2022-01-05 09:36:56,366 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / DEBUG - Message received: {'researcher_id': 'researcher_6614eef6-5331-401a-b4de-4e2a69e51c81', 'job_id': 'a538d4c6-01bc-48b5-beee-5b4db6b8e01f', 'training_args': {'batch_size': 20, 'lr': 1e-05, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 250}, 'model_args': {'num_class': 6}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/01/05/my_model_43380103-454c-42a3-bc34-13927bc21e33.py', 'params_url': 'http://localhost:8844/media/uploads/2022/01/05/researcher_params_65b83e7f-e1ae-4440-9d0b-06597b6743ab.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_de591a20-68ff-4697-8abb-9daccd955df2': ['dataset_e7cd0f44-0504-4bd3-b10d-ae41a7cc708c']}}
2022-01-05 09:36:56,368 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_6614eef6-5331-401a-b4de-4e2a69e51c81', 'job_id': 'a538d4c6-01bc-48b5-beee

2022-01-05 09:38:20,361 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / INFO - Train Epoch: 1 [1400/18000 (8%)]	Loss: 0.330827
2022-01-05 09:38:20,757 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d / INFO - Train Epoch: 1 [1400/16954 (8%)]	Loss: 0.475132
2022-01-05 09:38:21,580 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / INFO - Train Epoch: 1 [1400/18000 (8%)]	Loss: 0.312722
2022-01-05 09:38:34,719 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / INFO - Train Epoch: 1 [1600/18000 (9%)]	Loss: 0.270845
2022-01-05 09:38:35,060 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d / INFO - Train Epoch: 1 [1600/16954 (9%)]	Loss: 0.410416
2022-01-05 09:38:35,348 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / INFO - Train Epoch: 1 [1600/18000 (9%)]	Loss: 0.242465
2022-01-05 09:38:46,187 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / INFO 

2022-01-05 09:42:08,110 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / DEBUG - Reached 250 batches for this epoch, ignore remaining data
2022-01-05 09:42:08,780 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d / DEBUG - Reached 250 batches for this epoch, ignore remaining data
2022-01-05 09:42:11,487 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / INFO - results uploaded successfully 
2022-01-05 09:42:12,079 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / INFO - results uploaded successfully 
2022-01-05 09:42:12,188 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d / INFO - results uploaded successfully 
2022-01-05 09:42:21,645 fedbiomed INFO - Downloading model params after training on node_b4095fb0-5dac-49f5-9af7-a478f8942a4a - from http://localhost:8844/media/uploads/2022/01/05/node_params_7976cff5-3e42-4622-85b1-51cc3b4ae93f.pt
2022-01-05 09:42:22,426 fedbiomed INFO - Downlo

2022-01-05 09:42:26,316 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d / DEBUG - [TASKS QUEUE] Item:{'researcher_id': 'researcher_6614eef6-5331-401a-b4de-4e2a69e51c81', 'job_id': 'a538d4c6-01bc-48b5-beee-5b4db6b8e01f', 'params_url': 'http://localhost:8844/media/uploads/2022/01/05/researcher_params_e3aef962-463a-4b40-b663-f944aa960217.pt', 'training_args': {'batch_size': 20, 'lr': 1e-05, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 250}, 'training_data': {'node_d562b808-8ed3-401b-8222-494283894e5d': ['dataset_f2c80a1e-24d7-4c88-977f-ceff1acc9009']}, 'model_args': {'num_class': 6}, 'model_url': 'http://localhost:8844/media/uploads/2022/01/05/my_model_43380103-454c-42a3-bc34-13927bc21e33.py', 'model_class': 'MyTrainingPlan', 'command': 'train'}
2022-01-05 09:42:28,121 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / INFO - {'monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x139a304f0>, 'batch_size': 20, 'lr': 1e-05, 'epochs'

2022-01-05 09:45:12,926 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d / INFO - Train Epoch: 1 [2600/16954 (15%)]	Loss: 0.082917
2022-01-05 09:45:24,278 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / INFO - Train Epoch: 1 [2800/18000 (16%)]	Loss: 0.286130
2022-01-05 09:45:24,874 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / INFO - Train Epoch: 1 [2800/18000 (16%)]	Loss: 0.177098
2022-01-05 09:45:25,525 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d / INFO - Train Epoch: 1 [2800/16954 (17%)]	Loss: 0.345563
2022-01-05 09:45:36,491 fedbiomed INFO - log from: node_de591a20-68ff-4697-8abb-9daccd955df2 / INFO - Train Epoch: 1 [3000/18000 (17%)]	Loss: 0.111264
2022-01-05 09:45:37,080 fedbiomed INFO - log from: node_b4095fb0-5dac-49f5-9af7-a478f8942a4a / INFO - Train Epoch: 1 [3000/18000 (17%)]	Loss: 0.156391
2022-01-05 09:45:37,721 fedbiomed INFO - log from: node_d562b808-8ed3-401b-8222-494283894e5d /

## Testing


Once the federated model is obtained, it is possible to test it locally on an independent testing partition.
The test dataset is available at this link:

https://drive.google.com/file/d/1YbwA0WitMoucoIa_Qao7IC1haPfDp-XD/

In [9]:
!pip install gdown

In [10]:
import os
import shutil
import tempfile
import PIL
import torch
import numpy as np
from sklearn.metrics import classification_report

from monai.config import print_config
from monai.data import decollate_batch
from monai.metrics import ROCAUCMetric
from monai.networks.nets import DenseNet121
import zipfile
from monai.transforms import (
    Activations,
    AddChannel,
    AsDiscrete,
    Compose,
    LoadImage,
    RandFlip,
    RandRotate,
    RandZoom,
    ScaleIntensity,
    EnsureType,
)
from monai.utils import set_determinism

print_config()

MONAI version: 0.8.0
Numpy version: 1.19.1
Pytorch version: 1.10.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 714d00dffe6653e21260160666c4c201ab66511b

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 9.0.0
Tensorboard version: 2.7.0
gdown version: 4.2.0
TorchVision version: 0.11.2
tqdm version: 4.62.3
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: NOT INSTALLED or UNKNOWN VERSION.
pandas version: 1.3.5
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



Download the testing dataset on the local temporary folder.

In [11]:
import gdown
import zipfile

resource = "https://drive.google.com/uc?id=1YbwA0WitMoucoIa_Qao7IC1haPfDp-XD"
base_dir = tmp_dir_model.name 
test_file = os.path.join(base_dir, "MedNIST_testing.zip")

gdown.download(resource, test_file, quiet=False)

zf = zipfile.ZipFile(test_file)

for file in zf.infolist():
    zf.extract(file, base_dir)
    
data_dir = os.path.join(base_dir, "MedNIST_testing")

Downloading...
From: https://drive.google.com/uc?id=1YbwA0WitMoucoIa_Qao7IC1haPfDp-XD
To: /Users/mlorenzi/works/temp/fedbiomed/var/tmp/tmp8_zt0d5r/MedNIST_testing.zip
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9.50M/9.50M [00:00<00:00, 10.8MB/s]


Parse the data and create the testing data loader:

In [12]:
class_names = sorted(x for x in os.listdir(data_dir)
                     if os.path.isdir(os.path.join(data_dir, x)))
num_class = len(class_names)
image_files = [
    [
        os.path.join(data_dir, class_names[i], x)
        for x in os.listdir(os.path.join(data_dir, class_names[i]))
    ]
    for i in range(num_class)
]

num_each = [len(image_files[i]) for i in range(num_class)]
image_files_list = []

image_class = []
for i in range(num_class):
    image_files_list.extend(image_files[i])
    image_class.extend([i] * num_each[i])
num_total = len(image_class)
image_width, image_height = PIL.Image.open(image_files_list[0]).size

print(f"Total image count: {num_total}")
print(f"Image dimensions: {image_width} x {image_height}")
print(f"Label names: {class_names}")
print(f"Label counts: {num_each}")

Total image count: 6000
Image dimensions: 64 x 64
Label names: ['AbdomenCT', 'BreastMRI', 'CXR', 'ChestCT', 'Hand', 'HeadCT']
Label counts: [1000, 1000, 1000, 1000, 1000, 1000]


In [13]:
length = len(image_files_list)
indices = np.arange(length)
np.random.shuffle(indices)


test_split = int(0.1 * length)
test_indices = indices[:test_split]

test_x = [image_files_list[i] for i in test_indices]
test_y = [image_class[i] for i in test_indices]

val_transforms = Compose(
    [LoadImage(image_only=True), AddChannel(), ScaleIntensity(), EnsureType()])

y_pred_trans = Compose([EnsureType(), Activations(softmax=True)])
y_trans = Compose([EnsureType(), AsDiscrete(to_onehot=num_class)])

In [14]:
class MedNISTDataset(torch.utils.data.Dataset):
    def __init__(self, image_files, labels, transforms):
        self.image_files = image_files
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        return self.transforms(self.image_files[index]), self.labels[index]


test_ds = MedNISTDataset(test_x, test_y, val_transforms)
test_loader = torch.utils.data.DataLoader(
    test_ds, batch_size=300)

Define testing metric:

In [15]:
auc_metric = ROCAUCMetric()

To test the federated model we need to create a model instance and assign to it the model parameters estimated at the last federated optimization round.

In [16]:
model = exp.model_instance
model.load_state_dict(exp.aggregated_params[rounds - 1]['params'])

<All keys matched successfully>

Compute the testing performance:

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

y_true = []
y_pred = []
with torch.no_grad():
    for test_data in test_loader:
        test_images, test_labels = (
            test_data[0].to(device),
            test_data[1].to(device),
        )
        pred = model(test_images).argmax(dim=1)
        for i in range(len(pred)):
            y_true.append(test_labels[i].item())
            y_pred.append(pred[i].item())


In [18]:
print(classification_report(
    y_true, y_pred, target_names=class_names, digits=4))

              precision    recall  f1-score   support

   AbdomenCT     0.9630    0.9905    0.9765       105
   BreastMRI     0.9881    1.0000    0.9940        83
         CXR     1.0000    1.0000    1.0000        96
     ChestCT     0.9897    1.0000    0.9948        96
        Hand     1.0000    0.9815    0.9907       108
      HeadCT     1.0000    0.9732    0.9864       112

    accuracy                         0.9900       600
   macro avg     0.9901    0.9909    0.9904       600
weighted avg     0.9902    0.9900    0.9900       600



In spite of the relatively small training performed on the data shared in the 3 nodes, the performance of the federated model seems pretty good. Well done! 